In [1]:
import sys
import os
import numpy as np
import pandas as pd
import torch
from PIL import Image

import ipywidgets as widgets
from io import BytesIO

from ganpreffinder import GANPrefFinder
from jupyterGUI import GUIses
from jupyterComponentsExprGUI import GUICompExprSes

StyleGAN2: Optimized CUDA op FusedLeakyReLU not available, using native PyTorch fallback.
StyleGAN2: Optimized CUDA op UpFirDn2d not available, using native PyTorch fallback.


### Description:

Possible datasets: ffhq, car, cat, church, horse, bedrooms, kitchen, places

---

In [2]:
DEVICE = "cpu"
DEVICE2 = "cpu"
if torch.cuda.is_available():
    DEVICE = "cuda:0"
    DEVICE2 = "cuda:0"
print("Device used:", DEVICE)

Device used: cuda:0


In [3]:
""" Car dataset """
# (idx, start, end, strength, invert)
# 'Redness':          (22,  9, 11,   -8, False),
# 'Horizontal flip':  ( 0,  0,  5,  2.0, True),
# 'Add grass':        (41,  9, 11,  -18, False),
# 'Blocky shape':     (16,  3,  6,   25, False),
#component_layers_dict = {
#    0: (0, 5),
#    16: (3, 6),
#    22: (9, 11),
#    41: (9, 11)
#}
#component_indices_to_use = [0,16,22,41]




""" Faces dataset """
"""
    'frizzy_hair':             (31,  2,  6,  20, False),
    'background_blur':         (49,  6,  9,  20, False),
    'bald':                    (21,  2,  5,  20, False),
    'big_smile':               (19,  4,  5,  20, False),
    'caricature_smile':        (26,  3,  8,  13, False),
    'scary_eyes':              (33,  6,  8,  20, False),
    'curly_hair':              (47,  3,  6,  20, False),
    'dark_bg_shiny_hair':      (13,  8,  9,  20, False),
    'dark_hair_and_light_pos': (14,  8,  9,  20, False),
    'dark_hair':               (16,  8,  9,  20, False),
    'disgusted':               (43,  6,  8, -30, False),
    'displeased':              (36,  4,  7,  20, False),
    'eye_openness':            (54,  7,  8,  20, False),
    'eye_wrinkles':            (28,  6,  8,  20, False),
    'eyebrow_thickness':       (37,  8,  9,  20, False),
    #'face_roundness':          (37,  0,  5,  20, False),
    #'fearful_eyes':            (54,  4, 10,  20, False),
    #'hairline':                (21,  4,  5, -20, False),
    'happy_frizzy_hair':       (30,  0,  8,  20, False),
    'happy_elderly_lady':      (27,  4,  7,  20, False),
    'head_angle_up':           (11,  1,  4,  20, False),
    #'huge_grin':               (28,  4,  6,  20, False),
    'in_awe':                  (23,  3,  6, -15, False),
    #'wide_smile':              (23,  3,  6,  20, False),
    'large_jaw':               (22,  3,  6,  20, False),
    'light_lr':                (15,  8,  9,  10, False),
    #'lipstick_and_age':        (34,  6, 11,  20, False),
    'lipstick':                (34, 10, 11,  20, False),
    'mascara_vs_beard':        (41,  6,  9,  20, False),
    'nose_length':             (51,  4,  5, -20, False),
    #'elderly_woman':           (34,  6,  7,  20, False),
    #'overexposed':             (27,  8, 18,  15, False),
    'screaming':               (35,  3,  7, -15, False),
    'short_face':              (32,  2,  6, -20, False),
    'show_front_teeth':        (59,  4,  5,  40, False),
    'smile':                   (46,  4,  5, -20, False),
    #'straight_bowl_cut':       (20,  4,  5, -20, False),
    'sunlight_in_face':        (10,  8,  9,  10, False),
    'trimmed_beard':           (58,  7,  9,  20, False),
    'white_hair':              (57,  7, 10, -24, False),
    'wrinkles':                (20,  6,  7, -18, False),
    'boyishness':              (8,   2,  5,  20, False)
"""

component_layers_dict = {
    31:  (2,  6),
    49:  (6,  9),
    21:  (2,  5),
    19:  (4,  5),
    26:  (3,  8),
    33:  (6,  8),
    47:  (3,  6),
    13:  (8,  9),
    14:  (8,  9),
    16:  (8,  9),
    43:  (6,  8),
    36:  (4,  7),
    54:  (7,  8),
    28:  (6,  8),
    37:  (8,  9),
    30:  (0,  8),
    27:  (4,  7),
    11:  (1,  4),
    23:  (3,  6),
    22:  (3,  6),
    15:  (8,  9),
    34:  (10, 11),
    41:  (6,  9),
    51:  (4,  5),
    35:  (3,  7),
    32:  (2,  6),
    59:  (4,  5),
    46:  (4,  5),
    10:  (8,  9),
    58:  (7,  9),
    57:  (7, 10),
    20:  (6,  7),
    8:  (2,  5)
}
#component_indices_to_use = [31,49,21,19,26,33,47,13,14,16,43,36,54,28,37,30,27,11,23,22,15,34,41,51,35,32,59,46,10,58,57,20,8]
#component_indices_to_use = [8,10,11,19,20,21,26,31,32,33,46,54]
component_indices_to_use = [8,21,19,20,11,54,31]
#bald,boyness,big smile,wrinkles,head_angle_up,eye_openness,frizzy_hair

In [4]:
class_name = 'ffhq' #"car"
gan_sample_seed =  6293435 #440749230
strength_left_bound  = -10
strength_right_bound = 10

ganpref_usermodel = GANPrefFinder(class_name = class_name,
                    gan_sample_seed      = gan_sample_seed,
                    n_comp_in_use        = None,
                    comp_to_use          = component_indices_to_use,
                    comp_layers_dict   = component_layers_dict,
                    adaptive_components  = component_indices_to_use,
                    strength_left_bound  = strength_left_bound,
                    strength_right_bound = strength_right_bound,
                    acquisition_strategy = 'EI-EXT-FAST',
                    device=DEVICE,
                    skip_computations_during_initialization=False,
                    skip_xstaroptimization_during_initialization=False)
ganpref_nousermodel = GANPrefFinder(class_name = class_name,
                    gan_sample_seed      = gan_sample_seed,
                    n_comp_in_use        = None,
                    comp_to_use          = component_indices_to_use,
                    comp_layers_dict     = component_layers_dict,
                    adaptive_components  = component_indices_to_use,
                    strength_left_bound  = strength_left_bound,
                    strength_right_bound = strength_right_bound,
                    device=DEVICE2,
                    skip_computations_during_initialization=True,
                    skip_xstaroptimization_during_initialization=True)
#Strategies available: [PCD,EXT,RAND,EI,EI-FIXEDX,EXR,EI-EXT,EI-EXT-FAST,EI-VARMAX,EI-VARMAX-FAST,COORDINATE-VARMAX]

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1!

In [ ]:
# gui_comp_ses = GUICompExprSes(gpf)
# gui_comp_ses.run()

In [ ]:
guises = GUIses(ganpref_usermodel,ganpref_nousermodel)

In [ ]:
guises.run()

## Sample images from the distribution of preferred images (generated by user model)
### Run the cell below after the user session is completed.

In [ ]:
guises.run_preferred_image_sampler()

In [ ]:
#Continue